In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#define constants
#unrolled through 28 time steps
time_steps=100
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=12
#learning rate for adam
learning_rate=0.0001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=6
#size of batch
batch_size=35

In [3]:
train_data = pd.read_csv("train_data_400.csv").dropna()
test_data = pd.read_csv("test_data_400_1.csv")

In [4]:
# set(test_data["type_1"])

In [5]:
# train_data["type_1"] = train_data["user_id"].map(str) +"_"+ train_data["action_id"]

# Data Preprocessing

In [6]:
# dataTemp : CSV input/out 섞여있는 데이터 
# period : 자를 timeSteps
# onehot : default True, 원핫으로 return 할지 말지 결정
# return : 다 만들어진 X , Y
def dataPreprocessing(dataTemp , period , onehot = True , dropCol = True ):
    encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
    retY = pd.Series([])
#     retY = np.array([])
    retX = dataTemp
    temptype = -1
    cnt = 0
    dataTemp["type_1"] = dataTemp["user_id"].map(str) +"_"+ dataTemp["action_id"]
    for dataIter in range(dataTemp["action_id"].size//period):
        retY[cnt] = encoding[dataTemp["action_id"].iloc[dataIter*period][:3]]
        comp = dataTemp["type_1"].iloc[dataIter*period]
        if comp == temptype:
            cnt += 1
        else:
#             print(cnt)
            temptype = comp
            cnt += 1
#             retX = retX.drop(retX.index[dataIter:dataIter+period])
    if onehot:
        retY = pd.get_dummies(retY)
    if dropCol:
        retX = retX.drop(["action_id"] , axis = 1)
        retX = retX.drop(["user_id"] , axis = 1)
        retX = retX.drop(["time_50hz"] , axis = 1)
        retX = retX.drop(["type_1"] , axis = 1)
    return retX , retY
        

In [7]:
train_X , train_Y = dataPreprocessing(train_data , time_steps , True)
test_X , test_Y = dataPreprocessing(test_data , time_steps , True)

In [8]:
train_X.shape , train_Y.shape , test_X.shape , test_Y.shape

((126000, 12), (1260, 6), (17600, 12), (176, 6))

In [9]:
train_data.shape , test_data.shape

((126000, 16), (17600, 16))

In [10]:
train_data

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,user_id,action_id,time_50hz,type_1
0,3.058304,-1.227988,2.570999,0.027964,0.941814,0.334969,0.160508,-1.386834,-0.749713,0.204199,0.172657,-0.801048,1,jog_9,1,1_jog_9
1,3.075964,-1.225818,2.615277,0.022178,0.941083,0.337448,-0.217198,-0.612402,-0.682841,0.089974,-0.373914,-0.506332,1,jog_9,2,1_jog_9
2,3.103364,-1.235013,2.651791,0.012594,0.944152,0.329269,0.663253,-0.498534,-0.620223,0.260127,-0.364364,-0.781249,1,jog_9,3,1_jog_9
3,3.109208,-1.244901,2.678484,0.010366,0.947364,0.319989,0.458100,-1.202168,-0.304561,0.584253,-0.922813,-0.285169,1,jog_9,4,1_jog_9
4,3.074214,-1.263514,2.661371,0.020364,0.953159,0.301783,1.347809,-0.550578,0.610944,0.626501,-1.045978,-0.063884,1,jog_9,5,1_jog_9
5,2.994483,-1.301926,2.567805,0.038938,0.964072,0.262773,2.414809,1.069776,1.265800,0.580035,-1.048346,-0.200593,1,jog_9,6,1_jog_9
6,2.877705,-1.355284,2.426982,0.055779,0.976867,0.206445,3.261337,0.707698,1.036100,0.023216,-0.850921,-0.867426,1,jog_9,7,1_jog_9
7,2.595229,-1.423334,2.142191,0.076342,0.989147,0.125539,4.630348,-0.697815,1.018621,-0.063067,-0.196087,-0.033894,1,jog_9,8,1_jog_9
8,1.943180,-1.467603,1.509766,0.095950,0.994680,0.037479,4.045324,-1.028207,0.766063,0.011884,0.306009,0.265301,1,jog_9,9,1_jog_9
9,1.139627,-1.447025,0.703775,0.112156,0.992350,-0.051596,5.012080,-0.081093,0.972427,-0.363179,0.773397,0.642828,1,jog_9,10,1_jog_9


In [ ]:
# encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
# y_train = train_data["action_id"].apply(lambda x:encoding[x[:3]])
# y_test= test_data["action_id"].apply(lambda x:encoding[x[:3]])

In [ ]:
# y_train = pd.get_dummies(y_train)
# y_test = pd.get_dummies(y_test)

In [ ]:
# temp_input = train_data.drop(["action_id"] , axis = 1)
# temp_input = temp_input.drop(["user_id"] , axis = 1)
# temp_input = temp_input.drop(["time_50hz"] , axis = 1)
# test_input = test_data.drop(["action_id"] , axis = 1)
# test_input = test_input.drop(["user_id"] , axis = 1)
# test_input = test_input.drop(["time_50hz"] , axis = 1)

In [ ]:
# temp_X = temp_input[:1240503]
# temp_Y = y_train[:1240503]
# test_X = test_input[:170000]
# test_Y = y_test[:170000]

In [11]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder(tf.float32 ,[None,time_steps , n_input])
#input label placeholder
y=tf.placeholder(tf.int32 ,[None,n_classes])

In [12]:
input=tf.unstack(x ,time_steps,1)

In [13]:
#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=0.7)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [14]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [15]:
Yhat = tf.argmax(prediction,1)

In [16]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [17]:
#initialize variables
init=tf.global_variables_initializer()
test_X_temp = np.array(test_X).reshape(-1,time_steps , n_input)
epoch = 300
sess = tf.Session()

sess.run(init)
for ep  in range(1,epoch+1):    
    iter=0
    while iter<100:
        if (iter+1)*batch_size*time_steps > train_data.shape[0]:
#             print((iter)*batch_size*time_steps , train_data.shape[0])
            break
        batch_x = train_X[iter*batch_size*time_steps:(iter+1)*batch_size*time_steps]
        batch_y = train_Y[iter*batch_size:(iter+1)*batch_size]
        batch_x = np.array(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})  
        iter=iter+1
#         if iter % 10 == 0:
# #             acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
# #             los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
#             test_acc = sess.run(accuracy, feed_dict={x: test_X_temp, y: test_Y})
#             test_los = sess.run(loss, feed_dict={x: test_X_temp, y: test_Y})
#             print("iter {} ,  Test Acc {} , los {} ".format(iter,test_acc , test_los))
        
    if ep % 5 == 0:            
        test_acc = sess.run(accuracy, feed_dict={x: test_X_temp, y: test_Y})
        test_los = sess.run(loss, feed_dict={x: test_X_temp, y: test_Y})
        print("Epoch {} ,  Test Acc {} , los {} ".format(ep,test_acc , test_los))


Epoch 5 ,  Test Acc 0.2443181872367859 , los 1.7755237817764282 
Epoch 10 ,  Test Acc 0.33522728085517883 , los 1.5682892799377441 
Epoch 15 ,  Test Acc 0.4488636255264282 , los 1.3993254899978638 
Epoch 20 ,  Test Acc 0.4829545319080353 , los 1.2713652849197388 
Epoch 25 ,  Test Acc 0.5170454382896423 , los 1.1619386672973633 
Epoch 30 ,  Test Acc 0.5511363744735718 , los 1.0799044370651245 
Epoch 35 ,  Test Acc 0.5909090638160706 , los 1.0009334087371826 
Epoch 40 ,  Test Acc 0.6363636255264282 , los 0.9326955676078796 
Epoch 45 ,  Test Acc 0.6818181872367859 , los 0.8610983490943909 
Epoch 50 ,  Test Acc 0.6761363744735718 , los 0.797922670841217 
Epoch 55 ,  Test Acc 0.7159090638160706 , los 0.7581506371498108 
Epoch 60 ,  Test Acc 0.7386363744735718 , los 0.7445228695869446 
Epoch 65 ,  Test Acc 0.7386363744735718 , los 0.7765880823135376 
Epoch 70 ,  Test Acc 0.75 , los 0.7628254294395447 
Epoch 75 ,  Test Acc 0.7329545617103577 , los 0.7797483205795288 
Epoch 80 ,  Test Acc 0.75

In [18]:
# test_data_eval 에 분류를 위한 type_1 을 넣는데.
test_data_eval = test_data
test_data_eval["type_1"] = test_data["user_id"].map(str) +"_"+ test_data["action_id"]

In [19]:
# type 의 종류를 가지고 와서 나중에 loop 를 돌면서 accuracy 를 만들것임
dic_testType = list(set(test_data_eval["type_1"]))
# dic_testType

In [ ]:
# full loop 만들어보기


# 한 타입만 일단 가져와보고
tempX = test_data_eval[(test_data_eval["type_1"] == '2_wlk_15')]
# matrix 에 맞게 뒤에 잘라내줌.
tempX = tempX[:(tempX.shape[0] // time_steps)*time_steps]
# 여기에서 데이터를 X,Y 로 먼저 나눠준다. onehot 해뒀고 필요없는 column 까지 다 Drop
evalTestX , evalTestY = dataPreprocessing(tempX , time_steps , True , True)
evalTestX = np.array(evalTestX).reshape(-1,time_steps , n_input)
# one hot 에 하나만 있기때문에 나머지 다 넣어주고 순서 원래대로 변경
#         for i in range(6):
#             if i not in set(evalTestY):
#                 evalTestY[i] = 0
#         evalTestY=evalTestY[[0,1,2,3,4,5]]
print('predict : ' , sess.run(Yhat , feed_dict={x:evalTestX}) , " Real Y : " , evalTestY.columns)
#     print(evalTestX.shape , evalTestY.shape , ite)

In [22]:
# full loop 만들어보기
y_pred = []
y_true = []
# >>> accuracy_score(y_true, y_pred)
for ite in dic_testType:
    # 한 타입만 일단 가져와보고
    tempX = test_data_eval[(test_data_eval["type_1"] == ite)]
    # matrix 에 맞게 뒤에 잘라내줌.
    tempX = tempX[:(tempX.shape[0] // time_steps)*time_steps]
    # 여기에서 데이터를 X,Y 로 먼저 나눠준다. onehot 해뒀고 필요없는 column 까지 다 Drop
    evalTestX , evalTestY = dataPreprocessing(tempX , time_steps , True , True)
    evalTestX = np.array(evalTestX).reshape(-1,time_steps , n_input)
    # one hot 에 하나만 있기때문에 나머지 다 넣어주고 순서 원래대로 변경
#         for i in range(6):
#             if i not in set(evalTestY):
#                 evalTestY[i] = 0
#         evalTestY=evalTestY[[0,1,2,3,4,5]]
    yh = sess.run(Yhat , feed_dict={x:evalTestX})
    pre = Counter(yh).most_common(1)[0][0]
    tru = evalTestY.columns[0]
    y_pred.append(pre)
    y_true.append(tru)
    if(pre != tru):
      print("ite : " , ite , ' predict : ' , yh ,":" ,pre, " Real Y : " , tru)
#     print("ite : " , ite , ' predict : ' , yh , " Real Y : " , tru)
#     print('predict : ' , pre , " Real Y : " , tru)
#     print(evalTestX.shape , evalTestY.shape , ite)}

ite :  3_wlk_7  predict :  [2 3 3 2] : 2  Real Y :  3
ite :  1_std_14  predict :  [1 3 3 3] : 3  Real Y :  5
ite :  1_dws_1  predict :  [1 4 3 3] : 3  Real Y :  1
ite :  1_ups_3  predict :  [3 0 3 2] : 3  Real Y :  2
ite :  1_ups_12  predict :  [2 1 1 3] : 1  Real Y :  2
ite :  2_wlk_7  predict :  [2 2 2 2] : 2  Real Y :  3


In [21]:
accuracy_score(y_true, y_pred)

0.8636363636363636

In [ ]:

temp = test_data_eval[(test_data_eval["type_1"] == '2_dws_2')]
# reshape 이 자동으로 되게 time_steps 로 잘라줌
temp = temp[:(temp.shape[0] // time_steps)*time_steps]
xx , yy = dataPreprocessing(temp , time_steps , True , True)
xx = np.array(xx).reshape(-1,time_steps , n_input)
with tf.Session() as sess:
    sess.run(init)
    test_acc = sess.run(accuracy, feed_dict={x: xx, y: yy})
    test_los = sess.run(loss, feed_dict={x: xx, y: yy})

In [ ]:
for i in inputDic:
    temp = test_data_eval[(test_data_eval["type_1"] == i)]
    # reshape 이 자동으로 되게 time_steps 로 잘라줌
    temp = temp[:(temp.shape[0] // time_steps)*time_steps]
    xx , yy = dataPreprocessing(temp , time_steps , True , True)
    xx = np.array(xx).reshape(-1,time_steps , n_input)
    

In [ ]:
test_X_temp = np.array(test_X[:160000]).reshape(-1,time_steps , n_input)
with tf.Session() as sess:
    sess.run(init)
    test_acc = sess.run(accuracy, feed_dict={x: test_X_temp, y: test_Y[:1600]})
    test_los = sess.run(loss, feed_dict={x: test_X_temp, y: test_Y[:1600]})